In [ ]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np
from datetime import datetime
import os
import glob
from Model.imageProcess import f_image_read, f_image_preprocessing_010, f_save_image, f_imagesingle_read, f_image_box, f_image_box_license, f_image_box_license_2
from Model.sqlProcess import f_sql_object_detection

yolo_model = "yolo11x.pt"
YoloModelVehicle = YOLO(f"Yolo/{yolo_model}")
# Dapatkan mapping dari ID kelas ke nama kelas
class_names = YoloModelVehicle.names

# print(f"vehicle_results : {YoloModelVehicle.names}")

image_folder = 'Dataset\Vehicle'
image_files = glob.glob(os.path.join(image_folder, '*.jpg'))

for image in image_files:
    tanggal_start = datetime.today()
    print(f"image : {image}")

    frame = cv2.imread(image)

    vehicle_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    original_frame = frame.copy()

    # Deteksi kendaraan
    vehicle_results = YoloModelVehicle(frame)

    # print(f"vehicle_results : {vehicle_results}")
    for vehicle in vehicle_results:
        # print(f"vehicle : {vehicle.boxes}")
        for box in vehicle.boxes:
            # print(f"box : {box}")

            x1, y1, x2, y2 = map(int, box.xyxy[0].numpy())

            # print(f"box.cls[0] : {box.cls[0]}")

            class_id = int(box.cls[0])
            confidence = box.conf[0]

            if class_id in [2, 3, 5, 7] and confidence > 0.00:
                detected_class_label = class_names[class_id]

                # f_image_box(frame, x1, x2, y1, y2, "")

        # f_image_read(image1=original_frame, title1=f"{image} Original", image2=frame, title2="Object Box")

                tanggal_end = datetime.today()

                f_sql_object_detection(p_label_name=image, 
                                       p_yolo_model=yolo_model, 
                                       p_object_type=detected_class_label, 
                                       p_confidence=confidence, 
                                       p_start_dt=tanggal_start.strftime("%Y-%m-%d %H:%M:%S"), 
                                       p_end_dt=tanggal_end.strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
yolo_models = {'yolov10s.pt', 'yolov10n.pt', 'yolov10m.pt', 'yolov10b.pt', 'yolov10l.pt', 'yolov10x.pt', 'yolo11n.pt', 'yolo11s.pt', 'yolo11m.pt', 'yolo11l.pt', 'yolo11x.pt'}

for model in yolo_models:
    print(f"model : {model}") 